# Depencies + Imports

In [ ]:
!pip install openai

In [2]:
import openai
import json

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Create Embeddings Loop

In [4]:
#Change paths to downloaded datasets from https://doi.org/10.7910/DVN/TTP7AO
#OpenAI key can be found in presentation. NEVER push it to Github.
GDRIVE_PATH = '/content/drive/MyDrive/Hackathon: European Patent Office (EPO)/data/'
EMBEDDING_MODEL = 'text-embedding-ada-002'
COMPLETION_MODEL = 'text-davinci-003'
openai.organization_key = 'org-htZFqalTGYWyHwLcV1mL96KY'
openai.api_key = ''

# 1 file

In [5]:
CPCs = ['Y02C', 'Y02A', 'Y02B', 'Y02W', 'Y02D', 'Y02E', 'Y02P', 'Y02T', 'not_Y02']
for CPC in CPCs:
  with open(GDRIVE_PATH+f'ds_WO_{CPC}_complete_final_new.json', 'r') as file:
      dataset = json.load(file)
  break

In [9]:
export = []
for i, patent in enumerate(dataset):
  knowledge_graph_list = None
  
  try:
    openai.organization_key = 'org-htZFqalTGYWyHwLcV1mL96KY'
    #Change paths to downloaded datasets from https://doi.org/10.7910/DVN/TTP7AO
    openai.api_key = ''
    prompt_string = "Patent Claims:\n" + patent[1][:800] + '\nExtract a small knowledge graph from the patent claims in the format of a complete json list (max 10 triplets) of ["entity_1", "relation", "entity_2"], ... :'
    output_text = openai.Completion.create(
    engine="text-davinci-003",
    prompt=prompt_string,
    max_tokens= 1000,
    n=1,
    frequency_penalty=0,
    presence_penalty=0,
    temperature=0.7,
    stop= ']]'
    )
    knowledge_graph_list = json.loads(output_text["choices"][0]["text"].replace('\n',''))
  except:
    try: 
      openai.organization_key = 'org-htZFqalTGYWyHwLcV1mL96KY'
      #OpenAI key can be found in presentation. NEVER push it to Github.
      openai.api_key = ''
      prompt_string = "Patent Claims:\n" + patent[1][:800] + '\nExtract a small knowledge graph from the patent claims in the format of a complete json list (max 10 triplets) of ["entity_1", "relation", "entity_2"], ... :'
      output_text = openai.Completion.create(
      engine="text-davinci-003",
      prompt=prompt_string,
      max_tokens= 1000,
      n=1,
      frequency_penalty=0,
      presence_penalty=0,
      temperature=0.7,
      stop= ']]'
      )
      knowledge_graph_list = json.loads(output_text["choices"][0]["text"].replace('\n',''))

    except:
      pass

  if knowledge_graph_list:
    export += [[patent[1], knowledge_graph_list]]
  
  if i == 500:
    break

In [10]:
json.dump(export, open(f"{GDRIVE_PATH}relation_extraction_dataset.json", "w"))